In [13]:
import pandas as pd
import numpy as np

This code is to parse split the messed up session into normal sessions

In [97]:
run1 = run[run['Trial'] <= 60]
run2 = run[run['Trial'] > 60]

In [124]:
# Ensure the DataFrame is sorted by time
run1_marker = run1.sort_values('Time').reset_index(drop=True)

# Only keep rows with non-null labels
label_df = run1_marker[['Time', 'Label']].dropna().copy()

# Identify where the label changes (i.e., new group starts)
label_df['PrevLabel'] = label_df['Label'].shift()
label_df['LabelChange'] = label_df['Label'] != label_df['PrevLabel']

# Keep only the first row of each new group of the same label
first_in_group_df = label_df[label_df['LabelChange']].drop(columns=['PrevLabel', 'LabelChange'])

# Reset index for a clean DataFrame
run1_marker = first_in_group_df.reset_index(drop=True)

# Ensure the DataFrame is sorted by time
run2_marker = run2.sort_values('Time').reset_index(drop=True)

# Only keep rows with non-null labels
label_df = run2_marker[['Time', 'Label']].dropna().copy()

# Identify where the label changes (i.e., new group starts)
label_df['PrevLabel'] = label_df['Label'].shift()
label_df['LabelChange'] = label_df['Label'] != label_df['PrevLabel']

# Keep only the first row of each new group of the same label
first_in_group_df = label_df[label_df['LabelChange']].drop(columns=['PrevLabel', 'LabelChange'])

# Reset index for a clean DataFrame
run2_marker = first_in_group_df.reset_index(drop=True)

In [ ]:
run1_eeg = run1.drop(columns=['Label', 'OutcomeClass', 'Trial'], axis=1)

run2_eeg = run2.drop(columns=['Label', 'OutcomeClass', 'Trial'], axis=1)

In [128]:
run1_marker.to_csv('Data/CSV/Subj1031/sub-CLASS_SUBJ_1031_ses-S001ONLINE_FES_task-Default_run-001_eeg_markers.csv', index=False)
run2_marker.to_csv('Data/CSV/Subj1031/sub-CLASS_SUBJ_1031_ses-S001ONLINE_NOFES_task-Default_run-001_eeg_markers.csv', index=False)
run1_eeg.to_csv('Data/CSV/Subj1031/sub-CLASS_SUBJ_1031_ses-S001ONLINE_FES_task-Default_run-001_eeg.csv', index=False)
run2_eeg.to_csv('Data/CSV/Subj1031/sub-CLASS_SUBJ_1031_ses-S001ONLINE_NOFES_task-Default_run-001_eeg.csv', index=False)

code to load data

In [129]:
def run_parse(eeg_df, marker_df, FES):
    eeg_df['Label'] = np.nan

    # Ensure both time columns are sorted
    eeg_df = eeg_df.sort_values('Time')
    marker_df = marker_df.sort_values('Time').reset_index(drop=True)

    # Assign label to EEG based on marker intervals
    for i in range(len(marker_df)):
        t_start = marker_df.loc[i, 'Time']
        label = marker_df.loc[i, 'Label']
        
        if i < len(marker_df) - 1:
            t_end = marker_df.loc[i + 1, 'Time']
        else:
            t_end = np.inf

        eeg_df.loc[(eeg_df['Time'] >= t_start) & (eeg_df['Time'] < t_end), 'Label'] = label

    eeg_df['Label'] = eeg_df['Label'].dropna().astype(float).astype(int).astype(str)

    # Mapping for label → descriptive class name
    TRIGGERS = {
        "MI_BEGIN": "200",
        "MI_END": "220",
        "MI_EARLYSTOP": "240",
        "ROBOT_BEGIN": "300",
        "ROBOT_END": "320",
        "ROBOT_EARLYSTOP": "340",
        "ROBOT_CONFIRM_STOP": "345",
        "REST_BEGIN": "100",
        "REST_END": "120",
        "REST_EARLYSTOP": "140"
    }
    label_to_class = {v: k for k, v in TRIGGERS.items()}
    eeg_df['OutcomeClass'] = eeg_df['Label'].map(label_to_class)

    # Clean up duplicate consecutive classes
    eeg_df['PrevClass'] = eeg_df['OutcomeClass'].shift()
    events_clean = eeg_df[(eeg_df['OutcomeClass'] != eeg_df['PrevClass']) & (eeg_df['OutcomeClass'].notna())].copy()
    events_clean = events_clean[['Time', 'OutcomeClass']].reset_index(drop=True)

    # Get just the trial start events
    trial_begins = events_clean[events_clean['OutcomeClass'].isin(['MI_BEGIN', 'REST_BEGIN'])].reset_index(drop=True)

    # Assign trial numbers
    eeg_df['Trial'] = np.nan
    trial_num = 0

    for i in range(len(trial_begins)):
        t_start = trial_begins.loc[i, 'Time']
        
        if i < len(trial_begins) - 1:
            t_end = trial_begins.loc[i + 1, 'Time']
        else:
            t_end = np.inf  # last trial goes until the end of EEG data

        eeg_df.loc[(eeg_df['Time'] >= t_start) & (eeg_df['Time'] < t_end), 'Trial'] = trial_num
        trial_num += 1

    # Fill in any samples before the first MI/REST_BEGIN with trial -1 (optional)
    first_start = trial_begins['Time'].min()
    eeg_df.loc[eeg_df['Time'] < first_start, 'Trial'] = -1

    # Convert Trial to integer type
    eeg_df['Trial'] = eeg_df['Trial'].astype(int)
    
    eeg_df['FES'] = FES
    return eeg_df.drop(['PrevClass'], axis=1)


In [130]:
subjects = {'Subj1031': {'Online': {'Sessions': [{'Runs': []}, {'Runs': []}]}, 'Offline': {'Sessions': [{'Runs': []}, {'Runs': []}]}}, 'Subj1032': {'Online': {'Sessions': [{'Runs': []}, {'Runs': []}]}, 'Offline': {'Sessions': [{'Runs': []}, {'Runs': []}]}}, 'Subj1033': {'Online': {'Sessions': [{'Runs': []}, {'Runs': []}]}, 'Offline': {'Sessions': [{'Runs': []}, {'Runs': []}]}}}

In [131]:
subjects['Subj1031']['Online']['Sessions'][0]['Runs'].append(run_parse(pd.read_csv('Data/CSV/Subj1031/sub-CLASS_SUBJ_1031_ses-S001ONLINE_FES_task-Default_run-001_eeg.csv'), pd.read_csv('Data/CSV/Subj1031/sub-CLASS_SUBJ_1031_ses-S001ONLINE_FES_task-Default_run-001_eeg_markers.csv'), True))
subjects['Subj1031']['Online']['Sessions'][0]['Runs'].append(run_parse(pd.read_csv('Data/CSV/Subj1031/sub-CLASS_SUBJ_1031_ses-S001ONLINE_NOFES_task-Default_run-001_eeg.csv'), pd.read_csv('Data/CSV/Subj1031/sub-CLASS_SUBJ_1031_ses-S001ONLINE_NOFES_task-Default_run-001_eeg_markers.csv'), False))
subjects['Subj1031']['Online']['Sessions'][1]['Runs'].append(run_parse(pd.read_csv('Data/CSV/Subj1031/sub-CLASS_SUBJ_1031_ses-S002ONLINE_FES_task-Default_run-001_eeg.csv'), pd.read_csv('Data/CSV/Subj1031/sub-CLASS_SUBJ_1031_ses-S002ONLINE_FES_task-Default_run-001_eeg_markers.csv'), True))
subjects['Subj1031']['Online']['Sessions'][1]['Runs'].append(run_parse(pd.read_csv('Data/CSV/Subj1031/sub-CLASS_SUBJ_1031_ses-S002ONLINE_NOFES_task-Default_run-001_eeg.csv'), pd.read_csv('Data/CSV/Subj1031/sub-CLASS_SUBJ_1031_ses-S002ONLINE_NOFES_task-Default_run-001_eeg_markers.csv'), False))
subjects['Subj1031']['Offline']['Sessions'][0]['Runs'].append(run_parse(pd.read_csv('Data/CSV/Subj1031/sub-CLASS_SUBJ_1031_ses-S001OFFLINE_FES_task-Default_run-001_eeg.csv'), pd.read_csv('Data/CSV/Subj1031/sub-CLASS_SUBJ_1031_ses-S001OFFLINE_FES_task-Default_run-001_eeg_markers.csv'), True))
subjects['Subj1031']['Offline']['Sessions'][1]['Runs'].append(run_parse(pd.read_csv('Data/CSV/Subj1031/sub-CLASS_SUBJ_1031_ses-S002OFFLINE_NOFES_task-Default_run-001_eeg.csv'), pd.read_csv('Data/CSV/Subj1031/sub-CLASS_SUBJ_1031_ses-S002OFFLINE_NOFES_task-Default_run-001_eeg_markers.csv'), False))

In [133]:
subjects['Subj1032']['Online']['Sessions'][0]['Runs'].append(run_parse(pd.read_csv('Data/CSV/Subj1032/sub-CLASS_SUBJ_1032_ses-S001ONLINE_FES_task-Default_run-001_eeg.csv'), pd.read_csv('Data/CSV/Subj1032/sub-CLASS_SUBJ_1032_ses-S001ONLINE_FES_task-Default_run-001_eeg_markers.csv'), True))
subjects['Subj1032']['Online']['Sessions'][0]['Runs'].append(run_parse(pd.read_csv('Data/CSV/Subj1032/sub-CLASS_SUBJ_1032_ses-S001ONLINE_NOFES_task-Default_run-001_eeg.csv'), pd.read_csv('Data/CSV/Subj1032/sub-CLASS_SUBJ_1032_ses-S001ONLINE_NOFES_task-Default_run-001_eeg_markers.csv'), False))
subjects['Subj1032']['Online']['Sessions'][1]['Runs'].append(run_parse(pd.read_csv('Data/CSV/Subj1032/sub-CLASS_SUBJ_1032_ses-S002ONLINE_FES_task-Default_run-001_eeg.csv'), pd.read_csv('Data/CSV/Subj1032/sub-CLASS_SUBJ_1032_ses-S002ONLINE_FES_task-Default_run-001_eeg_markers.csv'), True))
subjects['Subj1032']['Online']['Sessions'][1]['Runs'].append(run_parse(pd.read_csv('Data/CSV/Subj1032/sub-CLASS_SUBJ_1032_ses-S002ONLINE_NOFES_task-Default_run-001_eeg.csv'), pd.read_csv('Data/CSV/Subj1032/sub-CLASS_SUBJ_1032_ses-S002ONLINE_NOFES_task-Default_run-001_eeg_markers.csv'), False))
subjects['Subj1032']['Offline']['Sessions'][0]['Runs'].append(run_parse(pd.read_csv('Data/CSV/Subj1032/sub-CLASS_SUBJ_1032_ses-S001OFFLINE_FES_task-Default_run-001_eeg.csv'), pd.read_csv('Data/CSV/Subj1032/sub-CLASS_SUBJ_1032_ses-S001OFFLINE_FES_task-Default_run-001_eeg_markers.csv'), True))
subjects['Subj1032']['Offline']['Sessions'][1]['Runs'].append(run_parse(pd.read_csv('Data/CSV/Subj1032/sub-CLASS_SUBJ_1032_ses-S002OFFLINE_NOFES_task-Default_run-001_eeg.csv'), pd.read_csv('Data/CSV/Subj1032/sub-CLASS_SUBJ_1032_ses-S002OFFLINE_NOFES_task-Default_run-001_eeg_markers.csv'), False))

In [135]:
subjects['Subj1033']['Online']['Sessions'][0]['Runs'].append(run_parse(pd.read_csv('Data/CSV/Subj1033/sub-CLASS_SUBJ_1033_ses-S001_ONLINE_FES_task-Default_run-001_eeg.csv'), pd.read_csv('Data/CSV/Subj1033/sub-CLASS_SUBJ_1033_ses-S001_ONLINE_FES_task-Default_run-001_eeg_markers.csv'), True))
subjects['Subj1033']['Online']['Sessions'][0]['Runs'].append(run_parse(pd.read_csv('Data/CSV/Subj1033/sub-CLASS_SUBJ_1033_ses-S001_ONLINE_NOFES_task-Default_run-001_eeg.csv'), pd.read_csv('Data/CSV/Subj1033/sub-CLASS_SUBJ_1033_ses-S001_ONLINE_NOFES_task-Default_run-001_eeg_markers.csv'), False))
subjects['Subj1033']['Online']['Sessions'][1]['Runs'].append(run_parse(pd.read_csv('Data/CSV/Subj1033/sub-CLASS_SUBJ_1033_ses-S002ONLINE_FES_task-Default_run-001_eeg.csv'), pd.read_csv('Data/CSV/Subj1033/sub-CLASS_SUBJ_1033_ses-S002ONLINE_FES_task-Default_run-001_eeg_markers.csv'), True))
subjects['Subj1033']['Online']['Sessions'][1]['Runs'].append(run_parse(pd.read_csv('Data/CSV/Subj1033/sub-CLASS_SUBJ_1033_ses-S002ONLINE_NOFES_task-Default_run-001_eeg.csv'), pd.read_csv('Data/CSV/Subj1033/sub-CLASS_SUBJ_1033_ses-S002ONLINE_NOFES_task-Default_run-001_eeg_markers.csv'), False))
subjects['Subj1033']['Offline']['Sessions'][0]['Runs'].append(run_parse(pd.read_csv('Data/CSV/Subj1033/sub-CLASS_SUBJ_1033_ses-S001_OFFLINE_FES_task-Default_run-001_eeg.csv'), pd.read_csv('Data/CSV/Subj1033/sub-CLASS_SUBJ_1033_ses-S001_OFFLINE_FES_task-Default_run-001_eeg_markers.csv'), True))
subjects['Subj1033']['Offline']['Sessions'][1]['Runs'].append(run_parse(pd.read_csv('Data/CSV/Subj1033/sub-CLASS_SUBJ_1033_ses-S002OFFLINE_NOFES_task-Default_run-001_eeg.csv'), pd.read_csv('Data/CSV/Subj1033/sub-CLASS_SUBJ_1033_ses-S002OFFLINE_NOFES_task-Default_run-001_eeg_markers.csv'), False))

trial data is from rest/mi begin to end, but this excludes most of the data. So, I just included all of the data in the trial (until the next begin). To only look at trial data:

In [75]:
trial_labels = {'REST_BEGIN', 'REST_END', 'MI_BEGIN', 'MI_END'}
trial_eeg_df = eeg_df[eeg_df['OutcomeClass'].isin(trial_labels)]
trial_eeg_df.shape

(437570, 44)